In [ ]:
import sys
import os
import pathlib
path_root = pathlib.Path(os.getcwd()).parents[1]
path_module = pathlib.Path(path_root) / 'chiral-client-python'
sys.path.append(str(path_module))
import chiral_client
print(f'chiral client version: {chiral_client.version}')
# register a user at cloud.chiral.one, get the token api from the page "Profile"
user_email = os.environ['CHIRAL_USER_EMAIL']
token_api = os.environ['CHIRAL_TOKEN_API']

In [ ]:
def create_client() -> chiral_client.Client:
    chiral_computing_host = "133.242.237.70"
    chiral_computing_port = "20000"
    options=[
        ('grpc.max_receive_message_length', 10 * 1000 * 1000),
    ]
    return chiral_client.Client(user_email, token_api, chiral_computing_host, chiral_computing_port, options)

cc = create_client()

In [ ]:
job_mgr = chiral_client.RecGenJobManager()
data_dir = os.environ['CHIRAL_DATA_DIR']
for (mol_file, result_count) in [
    # # 1-step transformation with fragment db of drug bank
    # ('sample1', 1117), 
    # ('sample4', 33),
    # # 1-step transformation with fragment db of kegg 
    # ('sample1', 1050), 
    # ('sample4', 44),
    # 2-step transformation with fragment db of drug bank 
    ('sample2', 80424), # 80784 from recgen doc 
    ('sample5', 14508), # 15210 from recgen doc
]:
    with open(f'{data_dir}/recgen/inputs/{mol_file}.mol') as f:
        input_mol = f.read()
        f.close()
    job_id = job_mgr.submit_job(cc, input_mol)
    cc.wait_until_completion(job_id)
    (output, error) = job_mgr.get_output(cc, job_id)
    print(len(output))
    print(error)
    # assert len(output) == result_count
    assert error == ''